### Project DRL : 
### Tree Chopping in Minecraft with Deep Q Learning from Demonstration
---
Made by : 
-  Kittichet Arriyathanasak Student No. 64340500005
-  Punyawat Prachongkij     Student No. 64340500040
          

## MineRL
---
MineRL is a research platform and dataset designed to support the development of artificial intelligence (AI) agents that can learn to complete complex tasks in the Minecraft game environment. It provides a large-scale dataset of human gameplay, including video frames, actions, and high-level goals, to help train and evaluate reinforcement learning and imitation learning algorithms. MineRL aims to make it easier for researchers to develop more efficient and generalizable AI systems by leveraging rich, diverse, and hierarchical data from a creative and open-ended environment like Minecraft.

#### MineRLTreechop-v0 environment
The MineRLTreechop-v0 environment is a Gym-compatible task within the MineRL suite that challenges an agent to chop down trees in a Minecraft world. The agent begins in a forested environment and must navigate to a tree, position itself correctly, and use the appropriate actions to break wood blocks. Observations include a first-person RGB image, inventory status, and other relevant data, while actions involve movement, camera control, and tool use. The reward structure encourages the agent to successfully collect wood, making it a relatively simple yet essential task that serves as a foundation for more complex behaviors in the MineRL framework.

#### Data Set
The MineRL Treechop-v0 dataset is a collection of human gameplay demonstrations for the tree-chopping task in Minecraft. It is part of the MineRL dataset and is used to train AI agents through imitation learning. The dataset includes sequences of player observations and actions as they move through the environment, locate trees, and chop wood. It provides a simple, focused task that helps researchers develop and test learning algorithms in a controlled setting before tackling more complex challenges in Minecraft.

#### Observation Space
1. Observation space of the mineRL enviroment is a game interface picture of the minecraft play with resulotion of 640x360 pixel contain with rgb color shape = ( 360,640,3 )
2. inventory status of the agent to check the reward ( obtain the wood log, reward + 1 )

<div style="text-align: center;">
  <img src="vpt.png" alt="Alt Text" width = 1280 height = 720/>
</div>


#### Action Space

Action space of the agent is the action same as the real game action space but convert into the discrete type action for less complexity of possible action. Note that Discrete and Box are actions spaces defined by Gym.

#### Action List
1. Move forward ( w key )
2. Turn camera with pitch = 0 degree and yaw = 5 degree
3. Attack ( left mouse click )
4. Turn camera with pitch = 5 degree and yaw = 0 degree
5. Turn camera with pitch = -5 degree and yaw = 0 degree
6. Turn camera with pitch = 0 degree and yaw = -5 degree
7. Jump foward ( w + space bar key )

#### Reward Term
This task encourages the agent to successfully collect wood log by

<div style="text-align: center;">

| Task | Reward Amount|
|--|--|
|Obtain wood log| +1|

</div>

### Assumption
---
The demonstration of expert action should guide the agent toward optimal action and should help the algorithm perform better than a pure RL-based method.
  
#### Why?
  
The Minecraft environment is a complex environment with a sequence of action that need to be taken before obtaining a reward. Especially in a treechop environment that a reward is sparse and there are multiple action required (navigate to a tree, hitting a block). If an agent can imitate the expert behaviour first then continuing to explore the environment by itself, an agent should converge to an optimal action much faster than purely RL based algorithm

## Experiment
---
then we would like to compare performance the algorithm with Pure Imitation Learning by the expert demonstration(Behavior Cloning), Pure Deep Reinforcement Learning(PPO) as a baseline and Combine both method as a implement algorithm(DQfD).

### Behavior Cloning ( Baseline )
---
#### Overview

**Behavior Cloning (BC)** is a type of **imitation learning** where an agent learns to perform tasks by mimicking expert behavior. Instead of learning through trial and error (like in reinforcement learning), the agent is trained using a dataset of demonstrations, typically collected from humans or other expert agents.


#### Principle

The core idea behind Behavior Cloning is to treat the problem like a **supervised learning** task. Given a dataset of state-action pairs — where each state represents an observation (e.g., image or sensor data) and each action is what the expert did in that state — the goal is to learn a policy that maps states to actions.

Formally:

- Input: $ s_t $ (state or observation at time $ t $)
- Output: $ a_t $ (action taken by the expert at time $ t $)

The agent learns a policy $ \pi(a|s) $ by minimizing the difference between the predicted actions and the expert actions in the dataset.


#### Implement with MineRLTreeChop-v0

1. Data Collection  
   - Download the mineRLTreeChop_v0 dataset that already provide the expert demonstraition

2. Supervised Training  
   - Use the dataset to train a model (CNN) that takes in an observation and outputs an action.
   - The model is trained using a Cross-Entropy Loss function.

3. Testing
   - After training, the model is used as a policy in the environment.
   - The agent acts by predicting the next action based on the current observation.


### Proximal Policy Optimization (PPO) ( Base line )
---
#### Overview

**Proximal Policy Optimization (PPO)** is a popular reinforcement learning algorithm developed by OpenAI. It is designed to improve the stability and performance of training policy-based agents. PPO strikes a balance between simplicity, efficiency, and reliable policy updates, making it one of the most widely used algorithms in deep reinforcement learning.

#### Principle

PPO is a **policy gradient method**, which means it directly optimizes the agent’s policy (i.e., how it chooses actions) by estimating gradients of expected rewards with respect to policy parameters.

Unlike older methods that could make large, unstable updates to the policy, PPO introduces a way to **limit how much the policy can change at each step**, ensuring updates are both effective and safe.

This is done using a **clipped surrogate objective**, which discourages big policy shifts by clipping the probability ratio between the new and old policy:
  
$$
L^{CLIP}(\theta) = \mathbb{E}_t \left[ \min \left( r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon) \hat{A}_t \right) \right]
$$

Where:
- $ r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)} $ is the probability ratio.
- $ \hat{A}_t $ is the advantage estimate.
- $ \epsilon $ is a small hyperparameter (e.g., 0.1–0.2).

#### How It Works

1. **Collect Trajectories**  
   - The agent interacts with the environment to collect a batch of experience: states, actions, rewards, and values.

2. **Compute Advantages**  
   - Use a method like **Generalized Advantage Estimation (GAE)** to estimate how much better an action is compared to average.

3. **Optimize Policy**  
   - Update the policy using the clipped surrogate loss. The clipping mechanism prevents the new policy from deviating too far from the old one.

4. **Repeat**  
   - The process repeats over many iterations, steadily improving the policy while maintaining stability.


### Deep Q-learning from Demonstrations (DQfD) ( Experiment Algorithm )

---
#### Overview

**Deep Q-learning from Demonstrations (DQfD)** is a reinforcement learning algorithm that combines **Q-learning** with **expert demonstrations** to improve learning efficiency and performance, especially in environments with sparse rewards. It extends the standard Deep Q-Network (DQN) by incorporating supervised learning from expert data alongside reinforcement learning from self-exploration.

#### Principle

The key idea of DQfD is to **leverage expert demonstrations** to guide the agent early in training, helping it avoid poor initial behavior and speeding up the learning process. It does this by training the agent on both:

- **Reinforcement signals** from the environment.
- **Supervised signals** from expert demonstrations.

DQfD introduces a **pretraining phase** using expert data and continues learning with both expert and agent-generated experiences.

The total loss function in DQfD is a combination of multiple components:

$$
\mathcal{L} = \mathcal{L}_{\text{DQN}} + \lambda_n \mathcal{L}_{n} + \lambda_e \mathcal{L}_{e}
$$

Where:
- $ \mathcal{L}_{\text{DQN}} $: Standard DQN loss (TD error).
- $ \mathcal{L}_{n} $: N-step return loss (for multi-step learning).
- $ \mathcal{L}_{e} $: Supervised large margin classification loss (to imitate expert actions).
- $ \lambda_n $ and $ \lambda_e $: Weighting hyperparameters.

#### How to implement with MineRLTreeChop-v0

1. Download Demonstration Data  
   - Use expert gameplay to build a dataset of state-action-reward sequences.

2. Pretrain the Agent 
   - Train the Q-network on demonstration data using a combination of temporal-difference (TD) loss and supervised imitation loss.

3. Start Reinforcement Learning  
   - The agent interacts with the environment and stores both its own and the demonstration data in a prioritized replay buffer.

4. Sample Mixed Batches
   - Each training batch includes a mix of demonstration and agent-generated experiences.

5. Train Using Combined Loss
   - Optimize the Q-network using the combined loss function, leveraging both reinforcement and imitation signals.

### Implementation
---
**Algorithm code path**
- `standalone/BC_PLUS_SCRIPT.py`: BC with train and test function.
- `standalone/RL_PLUS_SCRIPT.py`: PPO with train and test function.
- `forger_torch/main.py`: DQfD Algorithm with train and test function.

<b></b>

**Model code path**
- `standalone/BC_TRAIN_CARROT.pth`: BC Model.
- `standalone/PPO_TRAIN_CARROT.zip`: PPO Model.
- `forger_torch/DQFD_TRAIN_CARROT.zip`: DQfD models.

### Result
---
#### Behavior Cloning (BC)

Sampling batch size of 32 from the MineRLTreeChop_v0 dataset to imitation learning by using CNN that input as 64x64 pixel (down sampling) 3channel (RGB) and using the cross entropy loss function for update the model.  training with 36,000 steps

<div style="text-align: center;">

  <img src="img\BC\BC_TRAIN_LOSS.png" alt="Alt Text" width = 720 height = 480/>
  <!-- <img src="BC/BC_ACTION.png" alt="Alt Text" width = 640 height = 480/> -->
 
  Average 100 Episode Reward : 11.79 +- 6.50

  <img src="img\BC/Test_score.png" alt="Alt Text" width =1000 height = 480/>

</div>


#### PPO
Sampling batch size of 64 from the MineRLTreeChop_v0 dataset deep reignforcement learning by using PPO algorithm that input as 64x64 pixel (down sampling) 3channel (RGB) and using the ppo loss function for update the model. Training with 1,000 episodes. 

<div style="text-align: center;">

  <img src="img\PPO\mean_loss.png" alt="Alt Text" width = 640 height = 480/>
  <img src="img\PPO/value_loss.png" alt="Alt Text" width = 640 height = 480/>
  <img src="img\PPO\pg_loss.png" alt="Alt Text" width = 640 height = 480/>
  <img src="img\PPO/fraction_loss.png" alt="Alt Text" width = 640 height = 480/>
  <img src="img\PPO/kl_div.png" alt="Alt Text" width = 640 height = 480/>
  <img src="img\PPO/explain_var.png" alt="Alt Text" width = 640 height = 480/>

  Average 100 Episode Reward : 7.77 +- 10.82

  <img src="img\PPO/test_reward.png" alt="Alt Text" width = 1000 height = 480/>

</div>

#### DQfD

Sampling batch size of 32 from the MineRLTreeChop_v0 dataset with priority sampling using DQfD algorithm that input as 64x64 pixel (down sampling) 6channel (2 frame RGB) and using the DQfD loss function for update the model. Pre-Training with 100,000 update steps and Training with 100,000 update steps. 

##### Pre Train
<div style="text-align: center;">
  <img src="img/DQfD/Avg_Expert_Loss_DQfD.png" alt="Alt Text" width = 640 height = 480/><img src="img/DQfD/Avg_nStep_Loss_DQfD.png" alt="Alt Text" width = 640 height = 480/><img src="img/DQfD/Avg_td_Loss_DQfD.png" alt="Alt Text" width = 640 height = 480/><img src="img/DQfD/Avg_Loss_DQfD.png" alt="Alt Text" width = 640 height = 480/>
</div>

##### Training
<div style="text-align: center;">
  <img src="img/DQfD/Train_Avg_Expert_Loss_DQfD.png" alt="Alt Text" width = 640 height = 480/><img src="img/DQfD/Train_Avg_nStep_Loss_DQfD.png" alt="Alt Text" width = 640 height = 480/><img src="img/DQfD/Train_Avg_td_Loss_DQfD.png" alt="Alt Text" width = 640 height = 480/><img src="img/DQfD/Train_Avg_Loss_DQfD.png" alt="Alt Text" width = 640 height = 480/>
</div>

<div style="text-align: center;">

Average 100 Episode Reward : 1.64 +- 2.56

  <img src="img/DQfD/Test_Score.png" alt="Alt Text" width = 1000 height = 480/>
</div>

## Conclusion & Analysis
---
<div style="text-align: center;">

|Algorithm|Average Reward|
|-|-|
BC|11.79 ± 6.504|
PPO|7.77 ± 10.82|
DQfD(our)|1.64 ± 2.56|

</div>
  
From the result, we can see that the best performing algorithm is the Behavior Cloning (BC) followed by PPO then our implementation of DQfD. The agent trained on BC is able to perform at an average of 11.79 per episode which is around 12 logs while the dataset that provided as demonstration finished at 64 logs for every sequence. Given that the BC is a supervised learning, we expect that the model would be overfit to an expert data eventually and this is also reflected in the BC testing reward in which there is a high fluctuation of the reward in each episode. If the environment that the model get tested in bear no similarity to the dataset the model would failed eventually.

Another algorithm that seems to perform well enough is PPO. With the mean reward at 7.77 or around 8 logs per episode. If we disregard the low number of mean reward and focus on the max reward that the agent trained by PPO can achieved at 35 is much more higher than the max reward obtain by BC (25) or DQfD (10). 

The last one is our implementation of DQfD which has multiple interesting points. First, the pretrain process of our DQfD is as expected the expert loss decreased steadily as it is expected and the pretrained model is able to obtain a reward that is comparable to our BC agent. However, when we proceed to continue and train our agent with DQN the reward went down even though the TD loss and N-Step loss go down as expected. The final mean reward is at 1.64 which far from being considered successful in this environment. 

Focusing in the pretraining stage of DQfD. While the Expert loss and N-step TD loss gradually decrease, the single step TD loss wasn't in a same trend. This emphasize the complexity and sparseness of the environment since while the agent focus on imitating the expert behaviour. The TD loss just continue to increase. 

Given all these result, there are many aspect that this work can be improved. Firstly, the issue may lies im our DQfD replay buffer implementation since we exclude the expert data completely in the training phase. This might be the root cause that the agent performance get deteriorated when approaching the training phase. Another aspect is in the hyperparameter tuning and training duration. Due to time constraint, we are unable to train every algorithm with the scheduler to find an optimal hyperparameter selection. If we could train the RL-based algorithm with more training episode maybe the performance of these RL-algorithm could surpass the BC eventually. Lastly, the environment itself. Since the issue of sparse reward is affecting the RL-algorithm learning, by introducing a new denser reward term such as distance to the nearest tree although not entirely an observation that provide to the player normally could help improving the agent learning. 



## Reference
---
1. Deep Q-learning from Demonstrations, Todd Hester et al., (AAAI 2018), available at https://arxiv.org/abs/1704.03732

2. SEIHAI: A Sample-efficient Hierarchical AI for the MineRL Competition, Hangyu Mao et al. (the third International Conference on Distributed Artificial Intelligence, 2018), available at https://arxiv.org/abs/2111.08857

3. Playing Minecraft with Behavioural Cloning, (NeurIPS2019, 2018), available at https://arxiv.org/abs/2005.03374